In [28]:
import pandas as pd
import string
import numpy as np
import simpletransformers

In [29]:
pip install simpletransformers

In [30]:
!git clone https://github.com/Navjot862/Courseera_analytics_tools.git

fatal: destination path 'Courseera_analytics_tools' already exists and is not an empty directory.


In [31]:
Review=pd.read_csv("/content/Courseera_analytics_tools/Sample_Review.csv")
Review=Review[['Review']]

In [32]:
##Convert everything to lower cases 
Review['Review']=Review['Review'].str.lower()

In [33]:
Proffessor=['professors','instructor',"ittner","bushee","lecturer","teacher"]

Courses=['autocad','3d printing','traditional tooling','economy','business','course','analytics','accounting','accounting analytics',
         'business analytics', 'business market analysis','advanced neurobiology']

Content=['lecture','videos','content','material','assignments','information']

In [34]:
Review_list=list(Review['Review'])

In [35]:
##Creating an empty data frame
Review_Data_Train=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

In [36]:
for i in range(len(Review_list)):
    Chk=pd.DataFrame(columns=['Sentence_Id','Word','Label'])
#     print(i)
    l2=Review_list[i].split()
    for j in range(len(l2)):
        Chk.at[j, 'Sentence_Id'] = i
        Chk.at[j, 'Word'] = l2[j]
#     print(Review_Data_Train.head(2))
    Review_Data_Train=pd.concat([Review_Data_Train,Chk])
    

In [12]:
Review_Data_Train['Label']='O'

Data=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

for m in np.unique(Review_Data_Train['Sentence_Id']):
#     print(m)
    first_course=0
    first_prof=0
    first_content=0
    Review_Data_Sentence=Review_Data_Train[Review_Data_Train['Sentence_Id']==m]

    for i in range(Review_Data_Sentence.shape[0]):
        
        ##Course
        
        if Review_Data_Sentence['Word'].iloc[i] in Courses and first_course==0:
            Review_Data_Sentence['Label'].iloc[i]='B-ORG'
            first_course=first_course+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Courses and first_course>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-ORG'
            first_course=first_course+1   
            
        ##Professor    
        
        if Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof==0:
            Review_Data_Sentence['Label'].iloc[i]='B-PER'
            first_prof=first_prof+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-PER'
            first_prof=first_prof+1
        
        ##Content
        
        if Review_Data_Sentence['Word'].iloc[i] in Content and first_content==0:
            Review_Data_Sentence['Label'].iloc[i]='B-MISC'
            first_content=first_content+1   
            
            
        elif Review_Data_Sentence['Word'].iloc[i] in Content and first_content>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-MISC'
            first_content=first_content+1  
            
    Data=pd.concat([Data,Review_Data_Sentence])        
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to b

In [13]:
Train_l1=[]
for x in range(20):
    Train_l1.append(x)
print(Train_l1)

Test_l1=[]
for x in range(20,25):
    Test_l1.append(x)
print(Test_l1)

eval_l1=[]
for x in range(25,30):
    eval_l1.append(x)
print(eval_l1)

Train_df=Data[Data['Sentence_Id'].isin(Train_l1)]
Test_df=Data[Data['Sentence_Id'].isin(Test_l1)]
eval_df=Data[Data['Sentence_Id'].isin(eval_l1)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24]
[25, 26, 27, 28, 29]


In [43]:
Train_df=Data

In [44]:
Train_df.columns=["sentence_id", "words", "labels"]
Test_df.columns=["sentence_id", "words", "labels"]
eval_df.columns=["sentence_id", "words", "labels"]

In [45]:
np.unique(Train_df['labels'])

array(['B-MISC', 'B-ORG', 'B-PER', 'I-MISC', 'I-ORG', 'I-PER', 'O'],
      dtype=object)

In [46]:
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
model_args = NERArgs(overwrite_output_dir=True,num_train_epochs=3)
# model_args.train_batch_size = 16
# model_args.evaluate_during_training = True

model = NERModel("roberta", "roberta-base", args=model_args,use_cuda=False)

# Train the model
model.train_model(Train_df)

# # Evaluate the model
# result, model_outputs, preds_list = model.eval_model(eval_df)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(12, 0.8704863103727499)

In [47]:
result

{'eval_loss': 0.15479318797588348,
 'f1_score': 0.0,
 'precision': 0.0,
 'recall': 0.0}

In [40]:
predictions, raw_outputs = model.predict(["It has good content, but there were missing classes to give more advices or clear any question students have, when grading peer review homeworks there were plenty of mistakes that would have been avoided if there were more video lectures or forums with teachers."])

In [48]:
predictions, raw_outputs = model.predict(["The course is fine as far as content. My complaint is that there is no one to contact regarding papers or questions. You are left to the other students for your grading and assignment completion, meaning that some of your assignments can hang there for weeks before ever getting reviewed."])

In [49]:
predictions

[[{'The': 'O'},
  {'course': 'O'},
  {'is': 'O'},
  {'fine': 'O'},
  {'as': 'O'},
  {'far': 'O'},
  {'as': 'O'},
  {'content.': 'O'},
  {'My': 'O'},
  {'complaint': 'O'},
  {'is': 'O'},
  {'that': 'O'},
  {'there': 'O'},
  {'is': 'O'},
  {'no': 'O'},
  {'one': 'O'},
  {'to': 'O'},
  {'contact': 'O'},
  {'regarding': 'O'},
  {'papers': 'O'},
  {'or': 'O'},
  {'questions.': 'O'},
  {'You': 'O'},
  {'are': 'O'},
  {'left': 'O'},
  {'to': 'O'},
  {'the': 'O'},
  {'other': 'O'},
  {'students': 'O'},
  {'for': 'O'},
  {'your': 'O'},
  {'grading': 'O'},
  {'and': 'O'},
  {'assignment': 'O'},
  {'completion,': 'O'},
  {'meaning': 'O'},
  {'that': 'O'},
  {'some': 'O'},
  {'of': 'O'},
  {'your': 'O'},
  {'assignments': 'O'},
  {'can': 'O'},
  {'hang': 'O'},
  {'there': 'O'},
  {'for': 'O'},
  {'weeks': 'O'},
  {'before': 'O'},
  {'ever': 'O'},
  {'getting': 'O'},
  {'reviewed.': 'O'}]]

In [23]:
eval_df

,sentence_id,words,labels
0,25,the,O
1,25,instructor,B-PER
2,25,does,O
3,25,an,O
4,25,excellent,O
...,...,...,...
32,29,this,O
33,29,doesn't,O
34,29,reflect,O
35,29,her,O
